In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.pipeline import FeatureUnion
from sklearn.impute import SimpleImputer, MissingIndicator
import gc
import janestreet
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.pipeline import FeatureUnion
from sklearn.impute import SimpleImputer, MissingIndicator
import gc
import janestreet
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score,confusion_matrix,classification_report
from sklearn.model_selection import learning_curve,StratifiedKFold
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/jane-street-market-prediction/train.csv')


In [ ]:
print('train shape is {}'.format(train_df.shape))


In [ ]:
train_df.head()

In [ ]:
train_df = train_df[train_df['weight'] != 0]

train_df['action'] = ((train_df['weight'].values * train_df['resp'].values) > 0).astype('int')


X_train = train_df.loc[:, train_df.columns.str.contains('feature')]
y_train = train_df.loc[:, 'action']


In [ ]:
X_train = X_train.fillna(-999)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score,confusion_matrix,classification_report
from sklearn.model_selection import learning_curve,StratifiedKFold
from sklearn import preprocessing
import xgboost as xgb

print("XGBoost version:", xgb.__version__)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train,random_state=42)

# XGBoost Classifier

In [ ]:

clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=11,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    missing=-999,
    random_state=2020,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)


In [ ]:
clf.fit(X_train, y_train)

In [ ]:

y_pred1=clf.predict(X_test)

In [ ]:
#Accuracy on test split
clf.score(X_test, y_test)

In [ ]:
#Accracy on train split
clf.score(X_train, y_train)

In [ ]:
import seaborn as sns          
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred1).T
cm = cm.astype('float')/cm.sum(axis=0)
ax = sns.heatmap(cm, annot=True, cmap='Blues');
ax.set_xlabel('True Label',size=12)
ax.set_ylabel('Predicted Label',size=12)


# LogisticRegression

In [ ]:
clf = LogisticRegression(random_state=0,penalty='l2').fit(X_train, y_train)

In [ ]:
y_pred2=clf.predict(X_test)

In [ ]:
#Accuracy on test split
clf.score(X_test, y_test)

In [ ]:
#Accracy on train split
clf.score(X_train, y_train)

In [ ]:
import seaborn as sns          
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred2).T
cm = cm.astype('float')/cm.sum(axis=0)
ax = sns.heatmap(cm, annot=True, cmap='Blues');
ax.set_xlabel('True Label',size=12)
ax.set_ylabel('Predicted Label',size=12)

# Decision Tree Classifier


In [ ]:
from sklearn import tree

In [ ]:
clf = tree.DecisionTreeClassifier()

In [ ]:
clf.fit(X_train, y_train)


In [ ]:
y_pred3=clf.predict(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred3).T
cm = cm.astype('float')/cm.sum(axis=0)
ax = sns.heatmap(cm, annot=True, cmap='Blues');
ax.set_xlabel('True Label',size=12)
ax.set_ylabel('Predicted Label',size=12)

# SGDClassifier

In [ ]:

clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=100).fit(X_train, y_train)
y_hat = clf.predict(X_train)
y_hat_test = clf.predict(X_test)
score = accuracy_score(y_hat_test, y_test)


In [ ]:
score

In [ ]:
import seaborn as sns          
# Confusion Matrix
cm = confusion_matrix(y_test, y_hat_test).T
cm = cm.astype('float')/cm.sum(axis=0)
ax = sns.heatmap(cm, annot=True, cmap='Blues');
ax.set_xlabel('True Label',size=12)
ax.set_ylabel('Predicted Label',size=12)

# RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

In [ ]:
y_predR=clf.predict(X_test)

In [ ]:
#After training, check the accuracy using actual and predicted values.
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_predR))  

In [ ]:
import seaborn as sns          
# Confusion Matrix
cm = confusion_matrix(y_test, y_predR).T
cm = cm.astype('float')/cm.sum(axis=0)
ax = sns.heatmap(cm, annot=True, cmap='Blues');
ax.set_xlabel('True Label',size=12)
ax.set_ylabel('Predicted Label',size=12)